In [1]:
from qutip import *
from qutip.piqs import *
import numpy as np
import math
import matplotlib.pyplot as plt
import random

np.set_printoptions(threshold=np.inf)
qutip.settings.num_cpus = 8
qutip.about()


QuTiP: Quantum Toolbox in Python
Copyright (c) QuTiP team 2011 and later.
Current admin team: Alexander Pitchford, Nathan Shammah, Shahnawaz Ahmed, Neill Lambert, Eric Giguère, Boxi Li, Jake Lishman, Simon Cross and Asier Galicia.
Board members: Daniel Burgarth, Robert Johansson, Anton F. Kockum, Franco Nori and Will Zeng.
Original developers: R. J. Johansson & P. D. Nation.
Previous lead developers: Chris Granade & A. Grimsmo.
Currently developed through wide collaboration. See https://github.com/qutip for details.

QuTiP Version:      4.7.1
Numpy Version:      1.21.4
Scipy Version:      1.7.3
Cython Version:     0.29.33
Matplotlib Version: 3.7.1
Python Version:     3.9.16
Number of CPUs:     24
BLAS Info:          INTEL MKL
OPENMP Installed:   False
INTEL MKL Ext:      True
Platform Info:      Linux (x86_64)
Installation path:  /home/harsh/.local/lib/python3.9/site-packages/qutip
Please cite QuTiP in your publication.
For your convenience a bibtex reference can be easily generated u

In [2]:
def EncodedSt(M1, M2, N = 12, J = 5):
    j  = np.array([jj for jj in np.linspace(N/2,N%2/2,int(N//2+1))])
    mj = np.array([int(2*jj+1) for jj in j])
    dim = np.sum(mj)
    st_ind = np.sum(mj[:np.where(j==J)[0][0]])

    Log0 = np.zeros((dim,1),dtype=np.complex128)
    Log1 = np.zeros((dim,1),dtype=np.complex128)

    #Logical 0 state
    Log0[st_ind+J-M1] = 1
    
    #Logical 1 state
    Log1[st_ind+J-M2] = 1

    Log0 = Qobj(Log0)
    Log1 = Qobj(Log1)
    
    #Encoded state
    EncSt = np.sqrt(1/2)*Log0 + np.sqrt(1/2)*Log1

    return EncSt

In [3]:
## Effective Hamiltonian:
def Hamiltonians(Δ,γ,N=12):
    system = Dicke(N = N)
    [Jx, Jy, Jz] = jspin(N)
    Jp = jspin(N,'+')
    Jm = jspin(N,'-')
    
    He = Δ*Jz - (1j*γ/2)*(Jp*Jm)
    return He

In [4]:
## Monte-Carlo Evolution:
def Evolve(H,psi0,tmax,h,ϵ1,ϵ2,N,J,choice):
    st = []
    data = []
    Jp = jspin(N,'+')
    Jm = jspin(N,'-')
    #Jp = Jp/np.linalg.norm(Jp,ord='fro')
    
    st.append(psi0)
    #data.append(expect(Jz,psi0))
    t = 0;i=0;
    count=0
    tt = 0
    st_d = 0*psi0.copy()
    while t<=tmax:
        decSt = Jm*st[-1]
        dp = decSt.norm()
        if h*dp < ϵ1[i] :
            st_d = (1-1j*h*H.dag())*st[-1]
            if st_d.norm():
                st_d = st_d/st_d.norm()

        else:
            st_d = decSt.tidyup()
            #print(t)
            # Performing recovery
            if choice:
                st_d = Jp*st_d#Recovery(st_d,N,J)
                #print(np.nonzero(st_d)[0])
            if st_d.norm():
                st_d = st_d/st_d.norm()
            
            #print(count,t, np.nonzero(st_d)[0],[st_d[k] for k in np.nonzero(st_d)[0]])
            count+=1

        st.append(st_d)
        #data.append(expect(Jz,st_d))
        t += h
        i += 1
        if count==4:
                tt = t
    return st,tt

In [5]:
N = 20; J = 10;
tmax = 1; delta = 0.1/tmax;
dr = 0.5                                   #decay rate

EncSt1 = EncodedSt(9,-8,N,J)
EncSt2 = EncodedSt(10,-10,N,J)
Hwp = Hamiltonians(delta,dr,N)
Hnp = Hamiltonians(0,dr,N)

h = 0.003#/np.linalg.norm(Hwp, ord ='fro') # timestep
print('Time Step = ',h)

nmax = 10000; npts = int(tmax/h);
print('No of data points = ',npts)
print('No of trajectories = ',nmax)

t = np.linspace(0,tmax,npts+1)
print(np.nonzero(EncSt1))
print(np.nonzero(EncSt2))

Time Step =  0.003
No of data points =  333
No of trajectories =  10000
(array([ 1, 18]), array([0, 0]))
(array([ 0, 20]), array([0, 0]))


In [6]:
stt = ['Encoded','GHZ']
index = 0
for EncSt in [EncSt1,EncSt2]:
    print(f'Starting Calculations for {stt[index]} state')
    data = np.zeros((nmax,len(t)))
    dataec = np.zeros((nmax,len(t)))
    for ntraj in range(nmax):
        prob1 = np.random.uniform(0.0, 1.0, npts+1)    #Uniform random numbers
        prob2 = np.random.uniform(0.0, 1.0, npts+1)
            
        EncSt_np, tn = Evolve(Hnp,EncSt,tmax,h,prob1,prob2,N,J,0)
        EncSt_wp, tw = Evolve(Hwp,EncSt,tmax,h,prob1,prob2,N,J,0)
        EncSt_wpec, tne = Evolve(Hwp,EncSt,tmax,h,prob1,prob2,N,J,1)
        EncSt_npec, twe = Evolve(Hnp,EncSt,tmax,h,prob1,prob2,N,J,1)
        
    
        EncFid = [(1-fidelity(EncSt_np[i],EncSt_wp[i])) for i in range(len(t))]
        EncFidec = [(1-fidelity(EncSt_npec[i],EncSt_wpec[i])) for i in range(len(t))]
    
        data[ntraj,:] = EncFid
        dataec[ntraj,:] = EncFidec

        file = open(f'CollectiveData/{stt[index]}_Trajectory{ntraj}.txt','a+')
        for i in range(len(EncFid)):
            file.write(f'{t[i]}\t{data[ntraj,i]}\t{dataec[ntraj,i]}\n')
        file.close()
    
        if ntraj%100==99:
            print(f'Finished Calculations for {ntraj+1} trajectories.')
            
    DataEn =    np.array([np.average(data[:,j]) for j in range(len(t))])
    DataEc =    np.array([np.average(dataec[:,j]) for j in range(len(t))])
    DataEnStd = np.array([np.std(data[:,j],dtype=np.float64) for j in range(len(t))])
    DataEcStd = np.array([np.std(dataec[:,j],dtype=np.float64) for j in range(len(t))])
    
    file = open(f'Data_Collective{stt[index]}2_{N}_{J}_{nmax}.txt','a+')
    for i in range(len(DataEn)):
        file.write(f'{t[i]}\t{DataEn[i]}\t{DataEnStd[i]}\t{DataEc[i]}\t{DataEcStd[i]}\n')
    file.close()
    
    index += 1

Starting Calculations for Encoded state
Finished Calculations for 100 trajectories.
Finished Calculations for 200 trajectories.
Finished Calculations for 300 trajectories.
Finished Calculations for 400 trajectories.
Finished Calculations for 500 trajectories.
Finished Calculations for 600 trajectories.
Finished Calculations for 700 trajectories.
Finished Calculations for 800 trajectories.
Finished Calculations for 900 trajectories.
Finished Calculations for 1000 trajectories.
Finished Calculations for 1100 trajectories.
Finished Calculations for 1200 trajectories.
Finished Calculations for 1300 trajectories.
Finished Calculations for 1400 trajectories.
Finished Calculations for 1500 trajectories.
Finished Calculations for 1600 trajectories.
Finished Calculations for 1700 trajectories.
Finished Calculations for 1800 trajectories.
Finished Calculations for 1900 trajectories.
Finished Calculations for 2000 trajectories.
Finished Calculations for 2100 trajectories.
Finished Calculations fo

In [7]:
print(max(DataEn),max(DataEc))

0.004030254661699022 0.004030254661699057
